In [104]:
from datetime import datetime
from sqlalchemy import create_engine, inspect
from time import time

import json
import locale
import numpy as np
import pandas as pd
import requests

### TESTING

In [107]:
# dataset 2

url = "https://cer.gov.au/document/power-stations-and-projects-accredited"
df = pd.read_csv(url)
len(df)

280

### Database setup

Notes: put your postgresql configuration in the config.json file

In [111]:
with open('config.json') as config:
    db_config = json.load(config)

db_user = db_config['DB_USER']
db_password = db_config['DB_PASSWORD']
db_host = db_config['DB_HOST']
db_port = db_config['DB_PORT']
db_name = db_config['DB_NAME']

db_engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
gh_inspector = inspect(db_engine)

### Greenhouse and energy information by designated generation facility

In [114]:
gh_energy_data_facts = [
    {
        "time_period_start": datetime(2023, 1, 7),
        "time_period_stop": datetime(2024, 6, 30),
        "dataset_id": "ID0243"
    },
    {
        "time_period_start": datetime(2022, 1, 7),
        "time_period_stop": datetime(2023, 6, 30),
        "dataset_id": "ID0083"
    },
    {
        "time_period_start": datetime(2021, 1, 7),
        "time_period_stop": datetime(2022, 6, 30),
        "dataset_id": "ID0082"
    },
    {
        "time_period_start": datetime(2020, 1, 7),
        "time_period_stop": datetime(2021, 6, 30),
        "dataset_id": "ID0081"
    },
    {
        "time_period_start": datetime(2019, 1, 7),
        "time_period_stop": datetime(2020, 6, 30),
        "dataset_id": "ID0080"
    },
    {
        "time_period_start": datetime(2018, 1, 7),
        "time_period_stop": datetime(2019, 6, 30),
        "dataset_id": "ID0079"
    },
    {
        "time_period_start": datetime(2017, 1, 7),
        "time_period_stop": datetime(2018, 6, 30),
        "dataset_id": "ID0078"
    },
    {
        "time_period_start": datetime(2016, 1, 7),
        "time_period_stop": datetime(2017, 6, 30),
        "dataset_id": "ID0077"
    },
    {
        "time_period_start": datetime(2015, 1, 7),
        "time_period_stop": datetime(2016, 6, 30),
        "dataset_id": "ID0076"
    },
    {
        "time_period_start": datetime(2014, 1, 7),
        "time_period_stop": datetime(2015, 6, 30),
        "dataset_id": "ID0075"
    }
]

gh_energy_table_name = "greenhouse_and_energy"

gh_energy_column_aliases = [
    {"reporting_entity": ["reportingentity", "reportingEntity", "controllingcorporation"]},
    {"facility_name" : ["facilityname", "facilityName"]},
    {"type": ["type"]},
    {"state": ["state"]},
    {"electricity_production_gj": ["electricityproductionGJ", "electricityProductionGJ"]},
    {"electricity_production_mwh" :["electricityproductionMWh", "electricityProductionMWh", "electricityProductionMwh"]},
    {"total_scope_1_emissions_t_co2_e": ["totalscope1emissionstCO2e", "totalScope1EmissionstCO2e", "scope1tCO2e"]},
    {"total_scope_2_emissions_t_co2_e": ["totalscope2emissionstCO2e", "totalScope2EmissionstCO2e", "totalScope2EmissionstCO2e2", "scope2tCO2e"]},
    {"total_emissions_t_co2_e": ["totalemissionstCO2e", "totalEmissionstCO2e"]},
    {"emission_intensity_t_co2_emwh": ["emissionintensitytCO2eMWh", "emissionIntensitytCO2eMWh", "emissionIntensitytMwh"]},
    {"grid_connected": ["gridconnected", "gridConnected", "gridConnected2"]},
    {"grid": ["grid"]},
    {"primary_fuel": ["primaryfuel", "primaryFuel"]},
    {"important_notes": ["importantnotes", "importantNotes"]},
    {"time_period_start": ["time_period_start"]},
    {"time_period_stop": ["time_period_stop"]},
    {"dataset_id": ["dataset_id"]} 
]

In [116]:
def standardize_column_name(df, col_name_aliases):
    col_names = df.columns.tolist()
    col_map = {}

    for col in col_names:
        for alias_dict in col_name_aliases:
            alias = list(alias_dict.values())[0]
            if col in alias:
                col_map[col] = list(alias_dict.keys())[0]
                break

    return df.rename(columns=col_map)

total_row_inserted = 0
for index, fact in enumerate(gh_energy_data_facts):
    # read csv
    dataset_id = fact['dataset_id']
    url = f'https://api.cer.gov.au/datahub-public/v1/api/ODataDataset/NGER/dataset/{dataset_id}?select%3D%2A'
    response = requests.get(url)
    df = pd.DataFrame(response.json())

    # standardize column name
    df = standardize_column_name(df, gh_energy_column_aliases)

    # add columns: time_period, dataset_id
    df['time_period_start'] = fact['time_period_start']
    df['time_period_stop'] = fact['time_period_stop']
    df['dataset_id'] = fact['dataset_id']

    if index == 0:
        # Generate the CREATE TABLE statement
        create_table_statement = pd.io.sql.get_schema(df, gh_energy_table_name)
        
        # Print the generated statement
        print(create_table_statement)

        # Create table
        df.head(n=0).to_sql(name=gh_energy_table_name, con=db_engine, if_exists='replace')
    
    print(f'inserting table {fact['dataset_id']}...')
    t_start = time()
    df.to_sql(name=gh_energy_table_name, con=db_engine, if_exists='append')
    t_end = time()
    print(f'inserted table {fact['dataset_id']} in {t_end-t_start:10.3f} seconds')
    total_row_inserted = total_row_inserted + len(df)
    print(f'{total_row_inserted} rows inserted')
        

CREATE TABLE "greenhouse_and_energy" (
"reporting_entity" TEXT,
  "facility_name" TEXT,
  "type" TEXT,
  "state" TEXT,
  "electricity_production_gj" INTEGER,
  "electricity_production_mwh" INTEGER,
  "total_scope_1_emissions_t_co2_e" INTEGER,
  "total_scope_2_emissions_t_co2_e" REAL,
  "total_emissions_t_co2_e" INTEGER,
  "emission_intensity_t_co2_emwh" REAL,
  "grid_connected" TEXT,
  "grid" TEXT,
  "primary_fuel" TEXT,
  "important_notes" TEXT,
  "time_period_start" TIMESTAMP,
  "time_period_stop" TIMESTAMP,
  "dataset_id" TEXT
)
inserting table ID0243...
inserted table ID0243 in      0.089 seconds
775 rows inserted
inserting table ID0083...
inserted table ID0083 in      0.068 seconds
1480 rows inserted
inserting table ID0082...
inserted table ID0082 in      0.068 seconds
2171 rows inserted
inserting table ID0081...
inserted table ID0081 in      0.065 seconds
2826 rows inserted
inserting table ID0080...
inserted table ID0080 in      0.068 seconds
3447 rows inserted
inserting table ID

### Australian Bureau of Statistic Data

In [136]:
url = "https://www.abs.gov.au/methodologies/data-region-methodology/2011-24/14100DO0001_2011-24.xlsx"

#### Estimated resident population - Males - year ended 30 June												

In [139]:
df = pd.read_excel(url, sheet_name="Table 1", skiprows=6, usecols="A:C,M:AV")

# check table
df.head()

,Code,Label,Year,Males - 0-4 years (no.),Males - 5-9 years (no.),Males - 10-14 years (no.),Males - 15-19 years (no.),Males - 20-24 years (no.),Males - 25-29 years (no.),Males - 30-34 years (no.),...,Males - 40-44 years (%),Males - 45-49 years (%),Males - 50-54 years (%),Males - 55-59 years (%),Males - 60-64 years (%),Males - 65-69 years (%),Males - 70-74 years (%),Males - 75-79 years (%),Males - 80-84 years (%),Males - 85 and over (%)
0,AUS,Australia,2011.0,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,AUS,Australia,2016.0,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,AUS,Australia,2018.0,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,AUS,Australia,2019.0,799652,829903,803493,769691,894252,949985,934561,...,6.3,6.6,6,6.1,5.4,4.7,4.1,2.8,1.8,1.5
4,AUS,Australia,2020.0,789571,832349,824582,766260,877255,947862,946250,...,6.3,6.5,6.1,6,5.5,4.8,4.2,2.9,1.9,1.6


In [143]:
# clean column names
def clean_column_names(df):
    # Clean the column names
    df.columns = df.columns.str.lower()  # Convert to lowercase
    df.columns = df.columns.str.replace(' ', '_')  # Replace spaces with underscores
    df.columns = df.columns.str.replace('-', '_')  # Replace dash with underscores
    df.columns = df.columns.str.replace('no.', 'integer')  # Replace dash with underscores
    df.columns = df.columns.str.replace('%', 'pct')  # Replace % with 'pct'
    df.columns = df.columns.str.replace('[^a-z0-9_]', '', regex=True)  # Remove special characters
    return df

df = clean_column_names(df)    
df.columns

Index(['code', 'label', 'year', 'males___0_4_years_integer',
       'males___5_9_years_integer', 'males___10_14_years_integer',
       'males___15_19_years_integer', 'males___20_24_years_integer',
       'males___25_29_years_integer', 'males___30_34_years_integer',
       'males___35_39_years_integer', 'males___40_44_years_integer',
       'males___45_49_years_integer', 'males___50_54_years_integer',
       'males___55_59_years_integer', 'males___60_64_years_integer',
       'males___65_69_years_integer', 'males___70_74_years_integer',
       'males___75_79_years_integer', 'males___80_84_years_integer',
       'males___85_and_over_integer', 'males___0_4_years_pct',
       'males___5_9_years_pct', 'males___10_14_years_pct',
       'males___15_19_years_pct', 'males___20_24_years_pct',
       'males___25_29_years_pct', 'males___30_34_years_pct',
       'males___35_39_years_pct', 'males___40_44_years_pct',
       'males___45_49_years_pct', 'males___50_54_years_pct',
       'males___55_59_y

In [145]:
# convert data types
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

unchanged_col = ["code", "label", "year"]

# Opt-in to the future behavior of replace
pd.set_option('future.no_silent_downcasting', True)

for col in list(df.columns):
    if col not in unchanged_col:
        if "integer" in col:
            df[col] = df[col].replace('-', np.nan)
            df[col] = df[col].apply(lambda x: locale.atoi(str(x)) if isinstance(x, str) else x)
            df[col] = df[col].astype("Int64")
        else:
            df[col] = df[col].replace('-', np.nan)
            df[col] = df[col].apply(lambda x: locale.atof(str(x)) if isinstance(x, str) else x)

print(df.dtypes)

code                            object
label                           object
year                           float64
males___0_4_years_integer        Int64
males___5_9_years_integer        Int64
males___10_14_years_integer      Int64
males___15_19_years_integer      Int64
males___20_24_years_integer      Int64
males___25_29_years_integer      Int64
males___30_34_years_integer      Int64
males___35_39_years_integer      Int64
males___40_44_years_integer      Int64
males___45_49_years_integer      Int64
males___50_54_years_integer      Int64
males___55_59_years_integer      Int64
males___60_64_years_integer      Int64
males___65_69_years_integer      Int64
males___70_74_years_integer      Int64
males___75_79_years_integer      Int64
males___80_84_years_integer      Int64
males___85_and_over_integer      Int64
males___0_4_years_pct          float64
males___5_9_years_pct          float64
males___10_14_years_pct        float64
males___15_19_years_pct        float64
males___20_24_years_pct  

In [147]:
# check table
df.head()

,code,label,year,males___0_4_years_integer,males___5_9_years_integer,males___10_14_years_integer,males___15_19_years_integer,males___20_24_years_integer,males___25_29_years_integer,males___30_34_years_integer,...,males___40_44_years_pct,males___45_49_years_pct,males___50_54_years_pct,males___55_59_years_pct,males___60_64_years_pct,males___65_69_years_pct,males___70_74_years_pct,males___75_79_years_pct,males___80_84_years_pct,males___85_and_over_pct
0,AUS,Australia,2011.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AUS,Australia,2016.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AUS,Australia,2018.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AUS,Australia,2019.0,799652,829903,803493,769691,894252,949985,934561,...,6.3,6.6,6.0,6.1,5.4,4.7,4.1,2.8,1.8,1.5
4,AUS,Australia,2020.0,789571,832349,824582,766260,877255,947862,946250,...,6.3,6.5,6.1,6.0,5.5,4.8,4.2,2.9,1.9,1.6


In [149]:
# store data to database
table_name = 'estimated_resident_population_males'
df.to_sql(name=table_name, con=db_engine, if_exists='replace')

44